In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy.stats import norm

C:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


$$
d_1 = \frac{\ln(\frac{S}{K}) + (r + \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
d_2 = d_1 - s \cdot \sqrt{t} = \frac{\ln(\frac{S}{K}) + (r - \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

S - stock price

K - strike price

r - risk-free rate

stdev - standard deviation

T - time horizon (years)

To apply the black Szell's formula we won't need the PPF distribution we used to when we forecasted future stock prices.

Instead we will need the cumulative normal distribution.

--->>> The cumulative distribution shows how the data accumulates in time its output can never be below zero or above 1.

In [2]:
def d1(S, K, r, stdev, T):
    return (np.log(S / K) + (r + stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))
 
def d2(S, K, r, stdev, T):
    return (np.log(S / K) + (r - stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))

In [3]:
# Cumulative Distribution Function (cdf): shows how the data accumulates in time
# --> CDF takes as an argument a value from the data and will show us what portion of the data lies below that value
norm.cdf(0)

0.5

In [4]:
norm.cdf(0.25)

0.5987063256829237

In [5]:
norm.cdf(0.75)

0.7733726476231317

In [6]:
# Because it is expected to be the largest data point in our set
norm.cdf(9)

1.0

===>>> Black Scholes Merton:
$$
\textbf{C} = SN(d_1) - Ke^{-rt}N(d_2) 
$$

In [7]:
def BSM(S, K, r, stdev, T):
    return (S * norm.cdf(d1(S, K, r, stdev, T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

In [9]:
ticker = 'PG'  
data = pd.DataFrame()  
data[ticker] = wb.DataReader(ticker, data_source='yahoo', start='2007-1-1', end='2017-3-21')['Adj Close']
data

,PG
Date,
2007-01-03,43.147480
2007-01-04,42.819897
2007-01-05,42.452206
2007-01-08,42.545799
2007-01-09,42.438835
...,...
2017-03-15,82.880844
2017-03-16,82.917107
2017-03-17,82.518120


In [10]:
# Deliver the current stock price (the last adjusted closing price)
S = data.iloc[-1]
S

PG    82.690414
Name: 2017-03-21 00:00:00, dtype: float64

In [11]:
# Count daily return
log_returns = np.log(1 + data.pct_change())

In [12]:
# Count annual standard deviation
stdev = log_returns.std() * 250 ** 0.5
stdev

PG    0.17655
dtype: float64

In [13]:
# Calculate the price of the call option.
r = 0.025 # stick to a risk free rate of 2.5% corresponding to the yield of a 10 year government bond
K = 110.0 # assume the strike price equals $110 
T = 1 # assume the time horizon is 1 year

In [14]:
d1(S, K, r, stdev, T)

PG   -1.386525
dtype: float64

In [15]:
d2(S, K, r, stdev, T)

PG   -1.563076
dtype: float64

In [16]:
# Obtain the call option price ~ close to $1.68 
BSM(S, K, r, stdev, T)

PG    0.514584
Name: 2017-03-21 00:00:00, dtype: float64

In [ ]:
# Is it possible to have a call option price so much lower than the actual stock price.
# The value of the option depends on multiple parameters such as strike price timed immaturity and volatility.
# It is not directly proportional to the price of the security.
# Professionals implement a lot more complicated mathematical formulas and loops to specify the call within a millionth of its 
#value because when investing tens of millions infinitesimal differences matter.